# Data Modelling With Cassandra


#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

# ETL Pipeline for Pre-Processing the Files

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Getting the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Creating a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# joining the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# To get total number of rows 
#print(len(full_data_rows_list))
# To check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# Checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId




## Set Apache Cassandra Cluster

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb  
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

## The following are the three questions we need to know more about.

1. Get the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Create tables and queries to ask the following three questions of the data

### Query1:
    
<b>Get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
    
#### Data Modeling:
    
The combination of sessionId and itemInSession is unique, and we need results based on sessionId and itmeInSession, so these two columns could be the Composite Primary Key of the table.
    
Since itemInSession could be skewed to some specific value, it's better to use sessionId as Priamary key so the data could be evenly distributed on the nodes
    
We need the artist name, song title and song length based on the sessionId and itemInSession
Based on the above requirements, we could design the data model as follows:

```
Create table sessions(
    sessionId int,
    itemInSession int,
    artist_name text,
    song text,
    song_length float,
    PRIMARY KEY (sessionId, itemInSession) 
    ) WITH CLUSTERING ORDER BY (itemInSession DESC)

```

 <b>Create the table sessions

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_item \
        (sessionId int, itemInSession int, artist text, song text, length float, \
        PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                     

<b> Insert data into the table sessions

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]), ))

#### Run the test query

In [11]:
query = "SELECT artist, song, length FROM session_item \
         WHERE sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query 2:
#### Get only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    
<b> Data Modeling

Since we need results based on userid and sessionid, it could make good perfomance to use userid and sessionid as partition key to make sure the sessions belonging to the same user could be distributed to same node 2. Since the data needs to be sorted by itemInSession, the itemInSession should be the clustering key 3. Besides Primary Key, we need artist name, song title and user name for this query Based on the above requirements, we could design the data model as follows:
```
Create table song_play_list 
    ( 
        userid int,
        sessionid int,
        iteminsession int,
        firstname text,
        lastname text,      
        artist_name text,
        song text,     
        PRIMARY KEY((userid, sessionid), iteminsession)
     ) WITH CLUSTERING ORDER BY (iteminsession DESC)
```

#### Create the table song_playlist_session

In [ ]:
query = "CREATE TABLE IF NOT EXISTS names \
        (userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
        PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                       

#### Insert data into the table song_playlist_session

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO names (userId, sessionId, itemInSession, artist, \
                                                        song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assigning which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Run the test query

In [18]:
query = "SELECT artist, song, firstName, lastName FROM names \
         WHERE userId = 10 and sessionId = 182 \
         ORDER BY itemInSession DESC"

try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

for row in rows:
    print (row)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


### Query 3:
#### Get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

#### Data Modeling
Since we need the results based on song name, firstName and lastName might be not unique, we need to use song name and userid as Primary Key
Use the song name as partition key so the users data listened to the same song could be distributed on the same node, it will improve the query performance
we need firatName and lastName as well for this query
Based on the above requirements, we could design the data model as follows:
```
Create table users_playlist 
    (
        song text,
        user_id,<
        firstname text,
        lastname text,  
        PRIMARY KEY (song, user_id)  
   ) 
```

#### Create the table users_playlist

In [19]:
query = "CREATE TABLE IF NOT EXISTS all_hands \
        (song text, userId int, firstName text, lastName text, \
        PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)   

                    

#### Insert data into the table users_playlist

In [20]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO all_hands (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Run the test query

In [21]:
query = "SELECT firstName, lastName FROM all_hands \
         WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [22]:
session.execute("DROP TABLE session_item")
session.execute("DROP TABLE names")
session.execute("DROP TABLE all_hands")

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()